In [5]:
# coding: utf-8
import os
import sys

sys.path.append(os.pardir)  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
import numpy as np
import matplotlib.pyplot as plt
from deep_learning_github.dataset.mnist import load_mnist
from deep_learning_github.common.multi_layer_net import MultiLayerNet
from deep_learning_github.common.optimizer import SGD

(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True)

print(x_train.shape[0])
print(x_train.ndim)
print(x_train)

60000
2
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [4]:
permutation = np.random.permutation(x_train.shape[0])
permutation

array([19519, 58481, 11611, ...,  6299,  5290, 17334])

In [12]:
x = np.array([[1,2],[3,4]])
x.ndim

2

In [18]:
x = np.array([[[1,2],[3,4]]])
x.ndim
x[0,:,:,:]

IndexError: too many indices for array: array is 3-dimensional, but 4 were indexed

In [9]:
x = x_train[permutation, :]
print(len(x))

NameError: name 'x_tarin' is not defined